<a href="https://colab.research.google.com/github/ekgren/workshop/blob/main/Day1/Workshop_Primer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST, Maskininlärningens "Hello World!"

I första lab-delen av workshoppen ska vi gå igenom ett exempel med bildigenkänning: MNIST.

MNIST är ett dataset bestående av små bilder (28 x 28 pixlar) av siffror (0-9), med tillhörande sifferetikett (alltså siffran bilden föreställer). 

Datan består av 60.000 träningsexempel och 10.000 testexempel.

Vi börjar med att importera pythonpaket som kommer användas.

torch och torchvision är maskininlärningsbibliotek. 

*torch* är grundpaketet. Detta innehåller matematiska funnktioner, GPU-funktionalitet, ramverk för att definiera modeller, optimiera dessa m.m. och i synnerhet autograd: Ramverket som används för att beräkna gradienter.

*torchvision* är ett tillägg till torch för just bildhantering.

*matplotlib* är ett bibliotek för att visa diagram och bilder.

*tqdm* är ett bibliotek som visar "progress bars".

itertools är ett internt pythonbibliotek med hjälpfunktioner för iteratorer.


In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import tqdm
import itertools

Vi börjar med att ladda ner datan, och tranfsormera den till pytorch-tensorer. 
Datan består av 60.000 träningsexempel och 10.000 testexempel med svartvita bilder och etiketter.

In [ ]:
transform = torchvision.transforms.Compose(
    [
     torchvision.transforms.ToTensor(),
     ]
)

mnist_train = torchvision.datasets.MNIST(
    '/files', 
    train=True,
    download=True,
    transform=transform
)

mnist_test = torchvision.datasets.MNIST(
    '/files', 
    train=False,
    download=True,
    transform=transform
)

Datan är i form av *tensorer*: Alltså strukturerade flyttal. Nedan visualiserar vi hur det första träningsexemplet ser ut som en tensor (alltså en matris av 28 x 28 reella tal), i bildform, och vilken etikett bilden datapunkten.

In [ ]:
fig, ax = plt.subplots()

print('tensorform:')
for row in mnist_train[0][0][0]:
  print(' '.join(['{:.2f}'.format(val) for val in row]))

print('bildform:')
ax.imshow(mnist_train[0][0][0])
ax.set_yticks([])
ax.set_xticks([])

plt.show()

print('Etikett: {}'.format(mnist_train[0][1]))

Vi pröver med en enkel modell:

$y_d = b_d + \sum_{ij} pixel_{ij} \theta_{dij}$

Denna modell associerar varje pixel i bilden med en poäng per siffra, och summerar sedan ihop alla pixlars poänger. 

Vi implementerar modellen som en pytorch-modul. Pytorch-moduler förenklar 
koden genom att modulen ansvarar för parametrarna.

In [4]:
class Simple(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.theta = torch.nn.Parameter(torch.randn(10, 28, 28).div_(28))
    self.b = torch.nn.Parameter(torch.zeros(10))

  def forward(self, image):
    """
    Funktion som givet en bild ger en sannolikhetsdistribution över siffror.

    Varje enskild pixel associeras med varje siffra, för att få fram sifferpoängen 
    för en bild, multipliceras pixelvärden med motsvarande pixel-siffervärden och
    summeras sedan ihop.

    alltså för en bild:

    score[i] = b[i] = sum(image[h, w] * w[i, h, w] for h, w in [(0,0)..(28,28)])

    image: en "batch" bilder med form (B x 1 x 28 x 28)
    ger: en "batch" med sifferpoäng med form (B x 10)
    """

    pixel_digit_score = image * self.theta
    digit_score = self.b + pixel_digit_score.sum((2,3))
    return digit_score

  def loss(self, image, label):
    self(image)


In [5]:
BATCH_SIZE=32
EPOCHS = 2
DEVICE = 'cuda'

train_dataloader = torch.utils.data.DataLoader(
    mnist_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

test_dataloader = torch.utils.data.DataLoader(
    mnist_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Utvärdering

Nedan definieras utvärderingsförfarandet.

Funktionen tar en modell, går igenom all data i mnist-utvärderingsdatan, och returnerar den genomsnittliga lossen samt träffsäkerheten.

In [6]:
@torch.no_grad()
def eval_model(model):
  N = len(mnist_test)

  # Skapa en tqdm-progress bar
  batches = tqdm.tqdm(test_dataloader)

  # Initialiser statistik
  hits = torch.zeros(()).to(DEVICE)
  loss = torch.zeros(()).to(DEVICE)
  for image, label in batches:
    image = image.to(DEVICE)
    label = label.to(DEVICE)

    # Beräkna sifferpoängen enligt modellen
    scores = model(image)

    #Uppdatera loss och hits
    loss += torch.nn.functional.cross_entropy(scores, label, reduction='sum')
    hits += (scores.argmax(1) == label).sum()

  # Normalisera loss och hits så att vi får genomsnittlig loss och accuracy
  loss = (loss / N).item()
  acc = (hits / N).item()


  return loss, acc

# Träning

Nedan definieras ett träningsförfarande för en epok. 

Funktionen tar en modell och en optimerare, och går igenom all data i mnist-träningsdatan, och uppdaterar modellen enligt optimeraren. 

In [7]:

def train_epoch(model, optimizer):
  batches = tqdm.tqdm(train_dataloader)

  for ix, (image, label) in enumerate(batches):
    image = image.to(DEVICE)
    label = label.to(DEVICE)

    # Nollställ parametrarnas gradienter
    optimizer.zero_grad()

    # Beräkna sifferpoängen för bilderna.
    scores = model(image)

    # Givet Sifferpoängen och den riktiga siffran beräknas en loss.
    loss = torch.nn.functional.cross_entropy(scores, label)
    
    # Beräkna gradienten av lossen med avseende på modellens parametrar.
    loss.backward()

    # Uppdatera parametrarna
    optimizer.step()

    if ix % 10 == 0:
      batches.set_description('loss {:.2f}'.format(loss.item()))

# Träningsloop
Nedan är en träningsloop. 
Vi börjar med att initialisera en ny modell, och flytta den till GPUn.

Därefter skapar vi en optimerare, som ansvarar för att uppdatera modellens parametrar baserat på deras gradienter.

Loopen går ut på att gå igenom all data EPOCH gånger, och per epok träna modellen på alla träningsdata (*train_epoch*), samt utvärdera den på utvärderingsdatan (*eval_model*).

In [ ]:

#Initialisera en ny modell (och lägg den på GPUn)
model = Simple().to(device=DEVICE)

# SGD står för Stochastic Gradient Descent och är den enklaste gradient-baserade
# Optimeraren. Den tar helt enkelt ett steg i gradientens riktning viktat med
# lr (learning rate).

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

for epoch in range(EPOCHS):
  # Utvärdera modellen
  print('\nafter {} epochs, loss {:.2f}, accuracy: {:.1%}'.format(epoch, *eval_model(model)))

  # Träna modellen på all träningsdata
  train_epoch(model, optimizer)

#Utvärdera modellen
print('\nafter {} epochs, loss {:.2f}, accuracy: {:.1%}'.format(EPOCHS, *eval_model(model)))



# En mer invecklad modell

Nu när vi lyckats träna en enkel modell på MNIST kan vi gå vidare till att träna
en mer komplicerad modell. Tack vare pytorch är detta relativt enkelt att göra:
Vi behöver bara definiera en pytorch modul som beskriver någon funktion som mappar bild (i form av tensorer) till sifferpoänger.

Eftersom pytorch hanterar gradientberäkningarna, och vi har definierat vår funktion som en pytorch-modul, kan vi enkelt stoppa in dess parametrar i en optimerare: Träningsloopen ser exakt likadan ut. 

Den mer invecklade modellen är inte en särskilt bra bildmodell, men ett typexempel av vad man kan göra. Skrivet som vektorer och matrismultiplikationer gör modellen följande:
 

$\mathbf{h}_{l+1} = \max(\mathbf{\beta}_l + \mathbf{h}_l \mathbf{\theta}_l, 0)  $

$\mathbf{y} = \mathbf{\beta}_L + \mathbf{h}_L \mathbf{\theta}_L$

där $\mathbf{h}_0$ är bilden i tillplattad vektorform, och $\mathbf{h}_l+1$ är en vektor med aktiveringar för lager $l+1$. Antalet aktiveringar specificeras när man initialiserar modellen: Deep(12, 20) har två "gömda" lager, där det första har storlek 12, och de andra har storlek 20. 





In [10]:
class Deep(torch.nn.Module):
  """
  """
  def __init__(self, *hidden_dims):
    super().__init__()
    sizes = [28*28, *hidden_dims]

    inout = zip(sizes, sizes[1:])

    layers = []

    indim = 28*28

    for outdim in hidden_dims:
      layers.append(torch.nn.Linear(indim, outdim, bias=True))
      layers.append(torch.nn.ReLU())
      indim = outdim
    
    layers.append(torch.nn.Linear(indim, 10, bias=True))
    
    self.f = torch.nn.Sequential(*layers)
  
  def forward(self, image):
    (B, C, W, H) = image.shape
    return self.f(image.reshape(B, W*H))


In [ ]:
#Initialisera en ny modell (och lägg den på GPUn)

# Deep() är identisk med modellen "Simple", men man kan lägga till fler lager
# i modellen genom att instantiera den med Deep(h1, h2, h3, ...). 

#model = Deep().to(device=DEVICE)
#model = Deep(20).to(device=DEVICE)
model = Deep(20, 40, 20, 12).to(device=DEVICE)

print(model)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

# Adam är en mer invecklad optimerare, som oftast leder till bra resultat.
#optimizer = torch.optim.AdamW(model.parameters())

for epoch in range(EPOCHS):
  # Utvärdera modellen
  print('\nafter {} epochs, loss {:.2f}, accuracy: {:.1%}'.format(epoch, *eval_model(model)))

  # Träna modellen på all träningsdata
  train_epoch(model, optimizer)

#Utvärdera modellen
print('\nafter {} epochs, loss {:.2f}, accuracy: {:.1%}'.format(EPOCHS, *eval_model(model)))



I https://pytorch.org/docs/stable/nn.html# finns fler exempel på lager i neurala nätverk (likt torch.nn.Linear, eller torch.nn.ReLU). Det går också bra att göra beräkningar direkt, som i modellen Simple. 